In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [3]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, Normalizer, StopWordsRemover 
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf, col, isnan, isnull, broadcast, desc, lower, countDistinct, array_contains, array, count, avg, explode, split, lit, when
from pyspark.sql.types import FloatType, ArrayType, StringType 
import json 
import re
import pyspark.sql.functions as f
from pyspark.sql.functions import pandas_udf
from pyspark.ml.linalg import SparseVector

In [4]:
# Загрузка данных
items = spark.read.csv("/labs/slaba03/laba03_items.csv", header=True, inferSchema=True,
                          sep='\t', ignoreLeadingWhiteSpace=True).select(['item_id', 'content_type',
 'title',
 'year',
 'genres'])
train = spark.read.csv("/labs/slaba03/laba03_train.csv", header=True, inferSchema=True)
test = spark.read.csv("/labs/slaba03/laba03_test.csv", header=True, inferSchema=True)
views = spark.read.csv("/labs/slaba03/laba03_views_programmes.csv", header=True, inferSchema=True)

In [5]:
#Расчет признаков для телепередач
views = views.withColumn("duration", col("ts_end") - col("ts_start"))

# Время просмотра каждой передачи всеми пользователями
item_total_view_time = views.groupBy('item_id').agg(f.sum('duration').alias('item_total_view_time'))

# Количество уникальных пользователей, посмотревших передачу
item_unique_viewers = views.groupBy('item_id').agg(countDistinct('user_id').alias('item_unique_viewers'))

# Объединение признаков для передач
item_features = item_total_view_time.join(item_unique_viewers, on='item_id', how='left')

In [6]:
# Расчет признаков для пользователей
# Общее время просмотра передач пользователем
user_total_view_time = views.groupBy('user_id').agg(f.sum('duration').alias('user_total_view_time'))

# Количество передач, просмотренных пользователем
user_total_watched_items = views.groupBy('user_id').agg(countDistinct('item_id').alias('user_total_watched_items'))

# Объединение признаков для пользователей
user_view_features = user_total_view_time.join(user_total_watched_items, on='user_id', how='left')

In [7]:
# Доля покупок определенной передачи по train 
item_total_purchases = train.groupBy('item_id').agg(f.sum('purchase').alias('item_total_purchases'))

item_total_lines = train.groupBy('item_id').agg(count('purchase').alias('item_total_lines'))

# Соединение результатов
item_features = item_total_purchases.join(item_total_lines, on='item_id')

# Расчет доли покупок пользователя
item_features = item_features.withColumn('item_purchase_ratio', col('item_total_purchases') / col('item_total_lines'))

In [8]:
# Подсчет общего количества покупок пользователем

user_total_purchases = train.groupBy('user_id').agg(f.sum('purchase').alias('user_total_purchases'))

user_total_lines = train.groupBy('user_id').agg(count('purchase').alias('user_total_lines'))

# Соединение результатов
user_features = user_total_purchases.join(user_total_lines, on='user_id')

# Расчет доли покупок пользователя
user_features = user_features.withColumn('user_purchase_ratio', col('user_total_purchases') / col('user_total_lines'))

In [9]:
# Присоединение признаков пользователей к тренировочному датасету
train = train.join(user_view_features, on='user_id', how='left')
train = train.join(user_features, on='user_id', how='left')

# Присоединение признаков передач к тренировочному датасету
train = train.join(item_features, on='item_id', how='left')
train = train.join(item_total_view_time, on='item_id', how='left')
train = train.join(item_unique_viewers, on='item_id', how='left')

# Заполнение пропусков
train = train.na.fill(0)

# Проверка
#train.show(5)

In [10]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Выбор признаков и целевой переменной
feature_columns = [
    'user_total_view_time', 'user_total_watched_items', 'user_purchase_ratio',
    'item_total_view_time', 'item_unique_viewers', 'item_purchase_ratio'
]
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
train = assembler.transform(train)

In [11]:
train_df, test_df = train.limit(60000).randomSplit([0.7, 0.3])

In [12]:
# Обучение модели
gbt = GBTClassifier(labelCol='purchase', featuresCol='features', maxIter=10)
model = gbt.fit(train_df)

In [13]:
# Предсказание на тренировочных данных
predictions = model.transform(test_df)

# Оценка модели
evaluator = BinaryClassificationEvaluator(labelCol='purchase', rawPredictionCol='rawPrediction', metricName='areaUnderROC')
roc_auc = evaluator.evaluate(predictions)
print(f'ROC AUC: {roc_auc}')

ROC AUC: 0.9396469206313851


In [14]:
# Подготовка тестовых данных
test = test.join(user_total_view_time, on='user_id', how='left')
test = test.join(user_total_watched_items, on='user_id', how='left')
test = test.join(user_features, on='user_id', how='left')
test = test.join(item_total_view_time, on='item_id', how='left')
test = test.join(item_unique_viewers, on='item_id', how='left')
test = test.join(item_features, on='item_id', how='left')

test = test.na.fill(0)

In [15]:
test = assembler.transform(test)

In [16]:
# Предсказание
test_predictions = model.transform(test)

In [17]:
test_predictions.show(5)

+-------+-------+--------+--------------------+------------------------+--------------------+----------------+--------------------+--------------------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+----------+
|item_id|user_id|purchase|user_total_view_time|user_total_watched_items|user_total_purchases|user_total_lines| user_purchase_ratio|item_total_view_time|item_unique_viewers|item_total_purchases|item_total_lines| item_purchase_ratio|            features|       rawPrediction|         probability|prediction|
+-------+-------+--------+--------------------+------------------------+--------------------+----------------+--------------------+--------------------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+----------+
|   8389| 566758|    null|              447954|                     102|          

In [17]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType
import numpy as np

# Функция для извлечения вероятности положительного класса из DenseVector
def extract_probability(probability):
    return float(probability[1])

# Регистрация UDF
extract_probability_udf = udf(extract_probability, DoubleType())

# Применение UDF к колонке probability
result = test_predictions.withColumn('purchase', extract_probability_udf(col('probability')))

# Выбор необходимых колонок и сортировка
result = result.select('user_id', 'item_id', 'purchase').orderBy(['user_id', 'item_id'])

# Сохранение результатов
result.toPandas().to_csv('lab03.csv', index=False)